<a href="https://colab.research.google.com/github/Warziikx/training-datamining-mds/blob/master/20200424_13_RIO_final1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Vidéo de présentation](https://drive.google.com/file/d/1IojWBmV6ABrQ_bVxk87bgd3I95X8nBnF/view?usp=sharing)



**Step 1 :  **



In [0]:
!pip install plotly==4.6.0
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from plotly.subplots import make_subplots

In [0]:
data = pd.read_excel('https://population.un.org/wup/Download/Files/WUP2018-F01-Total_Urban_Rural.xls',skiprows = 29, header=None,)
data.columns=['Index', 'Name', 'Note', 'Country code', 'Urban Population', 'Rural population', 'Total Population', 'Urban percent']
data

,Index,Name,Note,Country code,Urban Population,Rural population,Total Population,Urban percent
0,13,AFRICA,NaN,903,547602.182,740318.336,1287920.518,42.518321
1,14,Eastern Africa,NaN,910,121315.521,312327.611,433643.132,27.975889
2,15,Burundi,NaN,108,1461.704,9754.746,11216.450,13.031788
3,16,Comoros,NaN,174,241.092,591.255,832.347,28.965323
4,17,Djibouti,NaN,262,755.533,215.875,971.408,77.777103
...,...,...,...,...,...,...,...,...
256,269,Samoa,NaN,882,36.066,161.629,197.695,18.243253
257,270,Tokelau,NaN,772,0.000,1.319,1.319,0.000000
258,271,Tonga,NaN,776,25.215,83.793,109.008,23.131330
259,272,Tuvalu,NaN,798,7.042,4.245,11.287,62.390361


In [0]:
countriesAndRegions = data[1].tolist()
countriesAndRegions
OrderedDatas = []
continents = []
numberOfContinents = 0
continentName = ''

for countrieAndRegion in countriesAndRegions:
  index = data[1].tolist().index(countrieAndRegion)
  if (data[3][index] != 5500 and data[3][index] != 5501):
    if (countrieAndRegion.isupper()):
      continent = {}
      continentName = countrieAndRegion.lower().capitalize()
      continent['name'] = continentName
      continent['urbanPop'] = data[4][index]
      continent['ruralPop'] = data[5][index]
      continent['percent'] = float(round(data[7][index], 1))
      continent['totalPop'] = data[6][index]
      continent['childs'] = []
      continents.append(continent)
    elif (data[3][index] >= 900):
      region = {}
      regionName = countrieAndRegion
      region['name'] = regionName
      region['urbanPop'] = data[4][index]
      region['ruralPop'] = data[5][index]
      region['percent'] = float(round(data[7][index], 1))
      region['totalPop'] = data[6][index]
      region['childs'] = []
      for continentFinal in continents:
        if (continentFinal.get('name') == continentName):
          continentChilds = continentFinal.get('childs')
          continentChilds.append(region)
    else :
      country = {} 
      country['name'] = countrieAndRegion
      country['urbanPop'] = data[4][index]
      country['ruralPop'] = data[5][index]
      country['totalPop'] = data[6][index]
      country['percent'] = float(round(data[7][index], 1))
      for continentFinal in continents:
        if (continentFinal.get('name') == continentName):
          regions = continentFinal.get('childs')
          if (continentFinal.get('name')  == 'Northern america'):
                regions.append(country)
          else:
            for regionFinal in regions:
              if (regionFinal.get('name') == regionName):
                regionChilds = regionFinal.get('childs')
                regionChilds.append(country)

**Format de sortie du tableau continents pour un continent**


```
[
  #continent
  {
    name: ,
    urbanPop: ,
    ruralPop: ,
    percent: ,
    childs: [
      #region
      {
        name: ,
        urbanPop: ,
        ruralPop: ,
        percent: ,
        childs: [
          #country
          {
            name: ,
            urbanPop: ,
            ruralPop: ,
            percent: ,
          }
        ]
      }
    ]
  }
]
```



In [0]:
characters = ["World"]
parents = [""]
values = [7632819.325]
percents = [55.3]
rural = [100 - 55.3]
for continent in continents:
  parents.append('World')
  characters.append(continent.get('name'))
  values.append(continent.get('totalPop'))
  percents.append(continent.get('percent'))
  rural.append(100 - continent.get('percent'))
  for region in continent.get('childs'):
    parents.append(continent.get('name'))
    characters.append(region.get('name'))
    values.append(region.get('totalPop'))
    percents.append(region.get('percent'))
    rural.append(100 - region.get('percent'))
    if (region.get('childs') is not None):
      for country in region.get('childs'):
        if((country.get('totalPop') /region.get('totalPop') * 100) < 10):
          if(region.get('name') + " smallest country" not in characters):
            parents.append(region.get('name'))
            characters.append(region.get('name') + " smallest country")
            smallTotalPop = 0
            nbOfSmallCountry = 0
            smallUrbanPercent = 0
            smallRuralPercent = 0
            for country2 in region.get('childs'):
              if((country2.get('totalPop') /region.get('totalPop') * 100) < 10):
                smallTotalPop += country2.get('totalPop')
                nbOfSmallCountry += 1
                smallUrbanPercent += country2.get('percent')
                smallRuralPercent += 100 - country2.get('percent')
            values.append(smallTotalPop)
            percents.append(float(round(smallUrbanPercent / nbOfSmallCountry,1)))
            rural.append(float(round(smallRuralPercent / nbOfSmallCountry,1)))
          parents.append(region.get('name') + " smallest country")
          characters.append(country.get('name'))
          values.append(country.get('totalPop'))
          percents.append(country.get('percent'))
          rural.append(100 - country.get('percent'))
        else:
          parents.append(region.get('name'))
          characters.append(country.get('name'))
          values.append(country.get('totalPop'))
          percents.append(country.get('percent'))
          rural.append(100 - country.get('percent'))




stubData = dict(
    parent = parents,
    character = characters,
    value = values,
    urban = percents,
    rural = rural
  )


**Présentation du graphique**

Ce graphique représente le pourcentage de population urbaine dans les différentes régions du monde.

Le graphique est représenté sous forme de Sunburst ou la division de chaque partie du graphique est faite par rapport à la population de la régiosn du monde.

Le graphique est divisé en 4 divisions : 

*   World
*   Contient
*   Région par contient
*   Pays

La couleur de chaque division varie selon le pourcentage de population urbaine.

**Utiisation du graphique**

Pour visualiser les différentes divisions il suffit de cliquer dessus. <br>
Pour revenir un cran plus haut de la division sélectionné sur suffit de cliquer sur le parent de cette même division. <br>
Pour avoir les détails de chaque division il suffit de passer le curseur de la souris sur la division voulue.

**Amélioration de l'expérience utilisateur**

Afin de représenter au mieux les différents pays et même mettre en avant ceux ayant une population faible, nous avons mis en place une divion supplémentaire au niveau des régions par continent. C'est-à-dire, lorsque vous êtes sur la dernières division et que vous avez accès à tous les pays de la région, une nouvelle catégorie a été crée regroupant tous les pays ayant moins de 10% de la population de la région. Ainsi cette amélioration permet de mieux visualier les plus petit pays contenu dans les mêmes régions des pays surpeuplés.





In [0]:
fig = make_subplots(1, 2, specs=[[{"type": "domain"}, {"type": "domain"}]],subplot_titles=("Urban", "Rural"))

fig.add_trace(go.Sunburst(
    labels=stubData['character'],
    parents=stubData['parent'],
    values=stubData['value'],
    branchvalues='total',
    marker=dict(
        colors=stubData['urban'],
        colorscale='rdylgn_r',
        cmid=55.3,
        colorbar=dict(bgcolor="#ffffff", title= "Color for percent", )
        ),
    hovertemplate='<b>%{label} </b> <br> Population: %{value} thousands<br> Urban Percent: %{color:.2f}',
    name='',
    maxdepth=2
    ), 1, 1)


fig.add_trace(go.Sunburst(
    labels=stubData['character'],
    parents=stubData['parent'],
    values=stubData['value'],
    branchvalues='total',
    marker=dict(
        colors=stubData['rural'],
        colorscale='rdylgn_r',
        cmid=55.3,
        # colorbar=dict(bgcolor="#ffffff", title= "Rural percent", )
        ),
    hovertemplate='<b>%{label} </b> <br> Population: %{value} thousands<br> Rural Percent: %{color:.2f}',
    name='',
    maxdepth=2
    ), 1, 2)


